In [34]:
import numpy as np 
import pandas as pd
import skimage.transform as tf
import glob, os, json



In [35]:
sub_folders = glob.glob('/mnt/HDD/jiwon/2024_spring_pilot/sub*')
offset_pre = []
offset_post = []

for sub_folder in sub_folders:
    calibration_folders = glob.glob(sub_folder +'/sub*_calibration*')
    sub_offset_pre = []
    sub_offset_post = []
    for data_dir in calibration_folders:
        gaze_vs_target = pd.read_csv(data_dir + '/gaze_vs_target.csv')

        # for each trial, grab the median x, y positions of one fifth of the data 
        trials = gaze_vs_target['trial id'].unique()
        gazes = []
        targets = []
        for t in trials:
            trial_data = gaze_vs_target.loc[gaze_vs_target['trial id']==t]
            trial_length = len(trial_data)
            gaze_pos = trial_data[['gaze x [deg]', 'gaze y [deg]']].iloc[-round(1/5):].median().values
            target_pos = trial_data[['target x [deg]', 'target y [deg]']].iloc[-round(1/5):].median().values
            gazes.append(gaze_pos)
            targets.append(target_pos)

        # compute offset before transformation 
        sub_offset_pre.append(np.mean(np.linalg.norm(np.array(gazes) - np.array(targets), axis=1)))

        # load parameter for transformation 
        params_dir = os.path.dirname(data_dir) + '/gaze_calibration_params.json'
        with open(params_dir, 'r') as file:
            params = json.load(file)

        # transform gazes 
        matrix_deg = tf.AffineTransform(matrix = params['tf_matrix_deg'])
        transformed = matrix_deg(gazes)

        # compute post transformation offset
        sub_offset_post.append(np.mean(np.linalg.norm(transformed - np.array(targets), axis=1)))
    
    # average within subject
    offset_pre.append(np.mean(sub_offset_pre))
    offset_post.append(np.mean(sub_offset_post))
    



In [37]:
np.mean(offset_post) - np.mean(offset_pre)

-1.3234157200158827